In [ ]:
# default_exp embeddings

# Embeddings
> AdaptNLP Embeddings Module

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import test_eq
from fastcore.xtras import is_listy

In [ ]:
#export
import logging
from typing import List, Dict, Union
from collections import defaultdict

from fastcore.dispatch import typedispatch
from flair.data import Sentence
from flair.embeddings import (
    Embeddings,
    WordEmbeddings,
    StackedEmbeddings,
    FlairEmbeddings,
    DocumentPoolEmbeddings,
    DocumentRNNEmbeddings,
    TransformerWordEmbeddings,
)

from adaptnlp.model_hub import FlairModelHub, HFModelHub, FlairModelResult, HFModelResult

/opt/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
#export
_flair_hub = FlairModelHub()
_hf_hub = HFModelHub()

In [ ]:
#export
logger = logging.getLogger(__name__)

In [ ]:
#exporti
@typedispatch
def _make_sentences(text:str, as_list=False) -> Union[List[Sentence], Sentence]:
    return [Sentence(text)] if as_list else Sentence(text)

In [ ]:
#hide
test_sentences = 'a,b,c'.split(',')
out = _make_sentences(test_sentences)
tst_out = [Sentence('a'), Sentence('b'), Sentence('c')]
for o,t in zip(out, tst_out):
    test_eq(o[0].text, t[0].text)

In [ ]:
#exporti
@typedispatch
def _make_sentences(text:list, as_list=False) -> Union[List[Sentence], Sentence]:
    if all(isinstance(t,str) for t in text):
        return [Sentence(t) for t in text]
    elif all(isinstance(t, Sentence) for t in text):
        return text

In [ ]:
#hide
test_sentence = 'My name is Zach'
out = _make_sentences(test_sentence, as_list=True)
tst_out = [Sentence(test_sentence)]
for o,t in zip(out, tst_out):
    test_eq(o[0].text, t[0].text)
test_eq(is_listy(out), True)

In [ ]:
#exporti
@typedispatch
def _make_sentences(text:Sentence, as_list=False) -> Union[List[Sentence], Sentence]:
    return [text] if as_list else text

In [ ]:
#hide
test_sentence = Sentence('Me')
out = _make_sentences(test_sentence)
test_eq(test_sentence[0].text, out[0].text)

In [ ]:
#exporti
@typedispatch
def _get_embedding_model(model_name_or_path:HFModelResult) -> TransformerWordEmbeddings:
    return TransformerWordEmbeddings(model_name_or_path.name)

In [ ]:
#exporti
@typedispatch
def _get_embedding_model(model_name_or_path:FlairModelResult) -> Union[FlairEmbeddings, WordEmbeddings]:
    nm = model_name_or_path.name
    try:
        return WordEmbeddings(nm.strip('flairNLP/'))
    except:
        return FlairEmbeddings(nm.strip('flairNLP/'))

In [ ]:
#exporti
@typedispatch
def _get_embedding_model(model_name_or_path:str) -> Union[TransformerWordEmbeddings, WordEmbeddings, FlairEmbeddings]:
    res = _flair_hub.search_model_by_name(model_name_or_path, user_uploaded=True)
    if len(res) < 1:
        # No models found
        res = _hf_hub.search_model_by_name(model_name_or_path, user_uploaded=True)
        if len(res) < 1:
            raise ValueError(f'Embeddings not found for the model key: {model_name_or_path}, check documentation or custom model path to verify specified model')
        else:
            return TransformerWordEmbeddings(res[0].name) # Returning the first should always be the non-fast option
    else:
        nm = res[0].name
        try:
            return WordEmbeddings(nm.strip('flairNLP/'))
        except:
            return FlairEmbeddings(nm.strip('flairNLP/'))

In [ ]:
#export
class EasyWordEmbeddings:
    """Word embeddings from the latest language models

    Usage:

    ```python
    >>> embeddings = adaptnlp.EasyWordEmbeddings()
    >>> embeddings.embed_text("text you want embeddings for", model_name_or_path="bert-base-cased")
    ```
    """

    def __init__(self):
        self.models: Dict[Embeddings] = defaultdict(bool)

    def embed_text(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
        model_name_or_path: Union[str, HFModelResult, FlairModelResult] = "bert-base-cased",
    ) -> List[Sentence]:
        """Produces embeddings for text

        **Parameters**:
        * `text` - Text input, it can be a string or any of Flair's `Sentence` input formats
        * `model_name_or_path` - The hosted model name key, model path, or an instance of either `HFModelResult` or `FlairModelResult`

        **Return**:
        * A list of Flair's `Sentence`s
        """
        # Convert into sentences
        sentences = _make_sentences(text)

        # Load correct Embeddings module
        if model_name_or_path not in self.models.keys():
            self.models[model_name_or_path] = _get_embedding_model(model_name_or_path)
        embedding = self.models[model_name_or_path]
        return embedding.embed(sentences)

    def embed_all(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
        *model_names_or_paths: str,
    ) -> List[Sentence]:
        """Embeds text with all embedding models loaded

        **Parameters**:
        * `text` - Text input, it can be a string or any of Flair's `Sentence` input formats
        * `model_names_or_paths` -  A variable input of model names or paths to embed

        **Return**:
        * A list of Flair's `Sentence`s
        """
        # Convert into sentences
        sentences = _make_sentences(text)

        if model_names_or_paths:
            for embedding_name in model_names_or_paths:
                sentences = self.embed_text(
                    sentences, model_name_or_path=embedding_name
                )
        else:
            for embedding_name in self.models.keys():
                sentences = self.embed_text(
                    sentences, model_name_or_path=embedding_name
                )
        return sentences

In [ ]:
#hide
import torch
embeddings = EasyWordEmbeddings()
res = embeddings.embed_text("text you want embeddings for", model_name_or_path="bert-base-cased")
test_eq(res[0][1].get_embedding().shape, torch.Size([768]))

Some weights of the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#hide
from transformers import pipeline
import timeit

from fastcore.test import test, test_close
import operator as op

# Ensure that speeds are comparible with that of raw HuggingFace

def performance_test(
    func_a:str, # A string of a function to time using AdaptNLP
    func_b:str, # A string of a function to time using HuggingFace,
    globals_a:dict, # A dict of globals `func_a` should expect
    globals_b:dict, # A dict of globals `func_b` should expect
    iterations:int=10, # Number of iterations
    repeat:int=5, # Number of times to repeat `number` iterations,
    thresh:float=0.01 # Threshold for `test_close`
):
    "Tests the performance of `func_a` vs `func_b` and checks they either are close, or a is faster"
    time_adapt = timeit.repeat(func_a, globals=globals_a, number=iterations, repeat=repeat)
    time_hf = timeit.repeat(func_b, globals=globals_b, number=iterations, repeat=repeat)
    avg_a, avg_b = sum(time_adapt)/repeat, sum(time_hf)/repeat
    # Test the performance
    try:
        test(avg_a, avg_b, op.lt)
    except:
        try:
            test_close(avg_a, avg_b, thresh)
        except Exception as e:
            raise Exception(f"""
            HuggingFace was comparitively faster than AdaptNLP. Functions used:
            -------------------------------------------------------------------
            
            AdaptNLP:
            \tFunction:{func_a}
            
            -------------------------------------------------------------------
            
            HuggingFace:
            \tFunction:{func_b}
            """)

In [ ]:
#hide
text = "text you want embeddings for"
pipe = pipeline('feature-extraction', 'bert-base-cased')
func_a = '_ = _embeddings.embed_text("text you want embeddings for", model_name_or_path="bert-base-cased")'
glob_a = {'_embeddings':embeddings}
func_b = '_ = _pipe(_text)'
glob_b = {'_pipe':pipe, '_text':text}
number=10

performance_test(func_a, func_b, glob_a, glob_b, iterations=number, thresh=0.1) # Keep thresh within a tenth of a second

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#export
class EasyStackedEmbeddings:
    """Word Embeddings that have been concatenated and "stacked" as specified by flair

    Usage:

    ```python
    >>> embeddings = adaptnlp.EasyStackedEmbeddings("bert-base-cased", "gpt2", "xlnet-base-cased")
    ```

    **Parameters:**

    * `&ast;embeddings` - Non-keyword variable number of strings specifying the embeddings you want to stack
    """

    def __init__(self, *embeddings: str):
        print("May need a couple moments to instantiate...")
        self.embedding_stack = []

        # Load correct Embeddings module
        for model_name_or_path in embeddings:
            self.embedding_stack.append(_get_embedding_model(model_name_or_path))

        assert len(self.embedding_stack) != 0
        self.stacked_embeddings = StackedEmbeddings(embeddings=self.embedding_stack)

    def embed_text(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
    ) -> List[Sentence]:
        """Stacked embeddings

        **Parameters**:
        * `text` - Text input, it can be a string or any of Flair's `Sentence` input formats

        **Return**:
        * A list of Flair's `Sentence`s
        """
        # Convert into sentences
        sentences = _make_sentences(text, as_list=True)

        # Unlike flair embeddings modules, stacked embeddings do not return a list of sentences
        self.stacked_embeddings.embed(sentences)
        return sentences

In [ ]:
#hide
embeddings = EasyStackedEmbeddings("bert-base-cased", "xlnet-base-cased")
sentences = embeddings.embed_text("This is Albert.  My last name is Einstein.  I like physics and atoms.")
test_eq(sentences[0][0].get_embedding().shape, torch.Size([1536]))

May need a couple moments to instantiate...


Some weights of the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if y

In [ ]:
#export
class EasyDocumentEmbeddings:
    """Document Embeddings generated by pool and rnn methods applied to the word embeddings of text

    Usage:

    ```python
    >>> embeddings = adaptnlp.EasyDocumentEmbeddings("bert-base-cased", "xlnet-base-cased", methods["rnn"])
    ```

    **Parameters:**

    * **`embeddings`** - Non-keyword variable number of strings referring to model names or paths
    * `methods` - A list of strings to specify which document embeddings to use i.e. ["rnn", "pool"] (avoids unncessary loading of models if only using one)
    * `configs` - A dictionary of configurations for flair's rnn and pool document embeddings
    ```python
    >>> example_configs = {"pool_configs": {"fine_tune_mode": "linear", "pooling": "mean", },
    ...                   "rnn_configs": {"hidden_size": 512,
    ...                                   "rnn_layers": 1,
    ...                                   "reproject_words": True,
    ...                                   "reproject_words_dimension": 256,
    ...                                   "bidirectional": False,
    ...                                   "dropout": 0.5,
    ...                                   "word_dropout": 0.0,
    ...                                   "locked_dropout": 0.0,
    ...                                   "rnn_type": "GRU",
    ...                                   "fine_tune": True, },
    ...                  }
    ```
    """

    __allowed_methods = ["rnn", "pool"]
    __allowed_configs = ("pool_configs", "rnn_configs")

    def __init__(
        self,
        *embeddings: str,
        methods: List[str] = ["rnn", "pool"],
        configs: Dict = {
            "pool_configs": {"fine_tune_mode": "linear", "pooling": "mean"},
            "rnn_configs": {
                "hidden_size": 512,
                "rnn_layers": 1,
                "reproject_words": True,
                "reproject_words_dimension": 256,
                "bidirectional": False,
                "dropout": 0.5,
                "word_dropout": 0.0,
                "locked_dropout": 0.0,
                "rnn_type": "GRU",
                "fine_tune": True,
            },
        },
    ):
        print("May need a couple moments to instantiate...")
        self.embedding_stack = []

        # Check methods
        for m in methods:
            assert m in self.__class__.__allowed_methods

        # Set configs for pooling and rnn parameters
        for k, v in configs.items():
            assert k in self.__class__.__allowed_configs
            setattr(self, k, v)

        # Load correct Embeddings module
        for model_name_or_path in embeddings:
            self.embedding_stack.append(_get_embedding_model(model_name_or_path))

        assert len(self.embedding_stack) != 0
        if "pool" in methods:
            self.pool_embeddings = DocumentPoolEmbeddings(
                self.embedding_stack, **self.pool_configs
            )
            print("Pooled embedding loaded")
        if "rnn" in methods:
            self.rnn_embeddings = DocumentRNNEmbeddings(
                self.embedding_stack, **self.rnn_configs
            )
            print("RNN embeddings loaded")

    def embed_pool(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
    ) -> List[Sentence]:
        """Generate stacked embeddings with `DocumentPoolEmbeddings`

        **Parameters**:
        * `text` - Text input, it can be a string or any of Flair's `Sentence` input formats

        **Return**:
        * A list of Flair's `Sentence`s
        """
        sentences = _make_sentences(text, as_list=True)
        self.pool_embeddings.embed(sentences)
        return sentences

    def embed_rnn(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
    ) -> List[Sentence]:
        """Generate stacked embeddings with `DocumentRNNEmbeddings`

        **Parameters**:
        * `text` - Text input, it can be a string or any of Flair's `Sentence` input formats

        **Return**:
        * A list of Flair's `Sentence`s
        """
        sentences = _make_sentences(text, as_list=True)
        self.rnn_embeddings.embed(sentences)
        return sentences

In [ ]:
#hide
embeddings = EasyDocumentEmbeddings("bert-base-cased", "xlnet-base-cased")
text = embeddings.embed_pool("This is Albert.  My last name is Einstein.  I like physics and atoms.")
test_eq(text[0].get_embedding().shape, torch.Size([1536]))

May need a couple moments to instantiate...
Pooled embedding loaded
RNN embeddings loaded


In [ ]:
#hide
text = embeddings.embed_rnn("This is Albert.  My last name is Einstein.  I like physics and atoms.")
test_eq(text[0].get_embedding().shape, torch.Size([512]))